In [1]:
import os
import json

from pymongo import MongoClient

# Prep

Set environment variables and setup simulated vasp run

In [2]:
#set the path to vasp pseudopotential files
#os.environ["VASP_PSP_DIR"] = os.path.join(os.getcwd(), "reference_files")

# Structure and input set

In [3]:
from pymatgen.util.testing import PymatgenTest
from pymatgen.io.vasp.sets import MPRelaxSet

struct_si = PymatgenTest.get_structure("Si")
vis = MPRelaxSet(struct_si, force_gamma=True)

# Define Fireworks and Workflow

In [4]:
from fireworks import Workflow
from matmethods.vasp.fireworks.core import OptimizeFW, StaticFW, NonSCFFW

fw1 = OptimizeFW(struct_si, vasp_input_set=vis, vasp_cmd=None, db_file=">>db_file<<")
fw2 = StaticFW(struct_si, vasp_input_set=vis, vasp_cmd=None, db_file=">>db_file<<", parents=[fw1])
fw3 = NonSCFFW(struct_si, vasp_cmd=None, db_file=">>db_file<<", parents=[fw2], mode="uniform")
fw4 = NonSCFFW(struct_si, vasp_cmd=None, db_file=">>db_file<<", parents=[fw2], mode="line")

fws = [fw1, fw2, fw3, fw4]

wf = Workflow(fws)

In [5]:
from matmethods.vasp.vasp_powerups import use_fake_vasp

#reference_dir = os.path.join(os.getcwd(), "reference_files")
reference_dir = os.path.abspath(os.path.join("..","..","matproj/MatMethods/matmethods/vasp/tests", 
                                             "reference_files"))

si_ref_dirs = {"structure optimization": os.path.join(reference_dir, "Si_structure_optimization"),
             "static": os.path.join(reference_dir, "Si_static"),
             "nscf uniform": os.path.join(reference_dir, "Si_nscf_uniform"),
             "nscf line": os.path.join(reference_dir, "Si_nscf_line")}

wf_fake = use_fake_vasp(wf, si_ref_dirs)

# Connect to Launchpad and reset

In [6]:
import datetime
from fireworks import LaunchPad

lp = LaunchPad.from_file("my_launchpad.yaml")
lp.reset(datetime.datetime.now().strftime('%Y-%m-%d'))

2016-08-03 07:53:34,234 INFO Performing db tune-up
2016-08-03 07:53:34,437 INFO LaunchPad was RESET.


# Submit Workflow

In [7]:
lp.add_wf(wf_fake)

2016-08-03 07:53:36,388 INFO Added a workflow. id_map: {-1: 4, -4: 1, -3: 2, -2: 3}


{-4: 1, -3: 2, -2: 3, -1: 4}

# Run

In [8]:
from fireworks import FWorker
from fireworks.core.rocket_launcher import rapidfire

db_file = os.path.join(os.getcwd(), "db.json")
rapidfire(lp, fworker=FWorker(env={"db_file": db_file}))

2016-08-03 07:53:38,050 INFO Created new dir /home/ray/Software/workshop-2016/matmethods/launcher_2016-08-03-14-53-38-050499
2016-08-03 07:53:38,051 INFO Launching Rocket
2016-08-03 07:53:38,101 INFO RUNNING fw_id: 4 in directory: /home/ray/Software/workshop-2016/matmethods/launcher_2016-08-03-14-53-38-050499
2016-08-03 07:53:38,106 INFO Task started: {{matmethods.vasp.firetasks.write_inputs.WriteVaspFromIOSet}}.
2016-08-03 07:53:38,132 INFO Task completed: {{matmethods.vasp.firetasks.write_inputs.WriteVaspFromIOSet}} 
2016-08-03 07:53:38,134 INFO Task started: {{matmethods.vasp.firetasks.run_calc.RunVaspFake}}.
2016-08-03 07:53:38,168 INFO matmethods.vasp.firetasks.run_calc RunVaspFake: verified inputs successfully
2016-08-03 07:53:38,180 INFO matmethods.vasp.firetasks.run_calc RunVaspFake: ran fake VASP, generated outputs
2016-08-03 07:53:38,181 INFO Task completed: {{matmethods.vasp.firetasks.run_calc.RunVaspFake}} 
2016-08-03 07:53:38,182 INFO Task started: {{matmethods.common.fire

# Retrieve and Verify

In [14]:
def get_task_collection(db_file):
    """
    connect to the database and return task collection
    """
    with open(db_file) as f:
        creds = json.loads(f.read())
        conn = MongoClient(creds["host"], creds["port"])
        db = conn[creds["database"]]
        if "admin_user" in creds:
            db.authenticate(creds["admin_user"], creds["admin_password"])
        return db[creds["collection"]]

In [15]:
from pymongo import DESCENDING

d1 = get_task_collection(db_file).find_one({"task_label": "structure optimization"}, sort=[("_id", DESCENDING)])

d2 = get_task_collection(db_file).find_one({"task_label": "static"}, sort=[("_id", DESCENDING)])

d3 = get_task_collection(db_file).find_one({"task_label": "nscf uniform"}, sort=[("_id", DESCENDING)])

d4 = get_task_collection(db_file).find_one({"task_label": "nscf line"}, sort=[("_id", DESCENDING)])

In [16]:
d1["output"]["bandgap"] # 0.65

0.6500000000000004

# Debug

In [13]:
!rm -rf launcher*